In [2]:
import os

In [2]:
from google.cloud import secretmanager

In [5]:
client = secretmanager.SecretManagerServiceClient()
NAME_SECRET = "projects/791883141034/secrets/TWITTER_BEARER_TOKEN/versions/1"
response = client.access_secret_version(request={"name": NAME_SECRET})



PermissionDenied: 403 Permission 'secretmanager.versions.access' denied for resource 'projects/791883141034/secrets/TWITTER_BEARER_TOKEN/versions/1' (or it may not exist).

# Query twitter API

In [11]:
import os
import requests
# from dotenv import load_dotenv, find_dotenv
from datetime import datetime
import pandas as pd
from dotenv import load_dotenv, find_dotenv


search_url = "https://api.twitter.com/2/tweets/search/recent"
# Optional params: start_time,end_time,since_id,until_id,max_results,next_token,
# expansions,tweet.fields,media.fields,poll.fields,place.fields,user.fields

def get_token():
    env_path = os.path.join(os.path.dirname(os.getcwd()),'.env')
    env_path = find_dotenv() # automatic find 
    load_dotenv(env_path)
    token = os.getenv('TWITTER_BEARER_TOKEN','Key missing in env settings')
    return token

def last_date_db():
    """
    Method to retrieve the last date in the DB
    """
    # Construct a BigQuery client object.

    query = """
        SELECT created_at
        FROM `wagon-bootcamp-802.my_dataset.twitter_table`
        ORDER BY created_at DESC
        LIMIT 1
    """
    df = pd.read_gbq(query, dialect='standard')
    return df.iloc[0]['created_at'].tz_localize(None).isoformat()+'Z'


def bearer_oauth(r):
    """
    Method required by bearer token authentication.
    """
    bearer_token = get_token()
    r.headers["Authorization"] = f"Bearer {bearer_token}"
    r.headers["User-Agent"] = "v2RecentSearchPython"
    return r

def connect_to_endpoint(url, params):
    response = requests.get(url, auth=bearer_oauth, params=params)
    print(response.status_code)
    if response.status_code != 200:
        raise Exception(response.status_code, response.text)
    return response.json()


def query_twitter(start_time=None):
    query_params = {'query': '#F1','tweet.fields':'created_at,lang', 'start_time':start_time, 'max_results' : '100'}
    json_response = connect_to_endpoint(search_url, query_params)
    return json_response

def func():
    '''
    Pushing results to GBQ
    '''
    most_recent_dt = last_date_db()
    df = pd.DataFrame(query_twitter(most_recent_dt)['data']).iloc[:-1]
    df = df[df['lang'] == 'en']
    df['created_at'] = pd.to_datetime(df['created_at'])
    table_id = 'wagon-bootcamp-802.my_dataset.new_table'
    df.to_gbq(table_id, if_exists='append')
    print('Done')
    return df

In [12]:
func()

/home/jeremymarchand/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2009: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


200


100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00, 11275.01it/s]

Done


,text,created_at,id,lang
0,RT @Motorsport: With Carlos Sainz starting fro...,2022-07-03 07:28:09+00:00,1543496803856908291,en
1,RT @F1: Just the Medical Car doing a side whee...,2022-07-03 07:28:03+00:00,1543496778892337152,en
2,RT @F1: That smile will be a bit bigger today!...,2022-07-03 07:28:02+00:00,1543496776010924035,en
3,RT @F1MikaHakkinen: Red pill or blue pill? #f1...,2022-07-03 07:27:58+00:00,1543496757203685377,en


In [9]:
most_recent_dt = last_date_db()
df = pd.DataFrame(query_twitter(most_recent_dt)['data']).iloc[:-1]
df = df[df['lang'] == 'en']
df['created_at'] = pd.to_datetime(df['created_at'])

/home/jeremymarchand/.pyenv/versions/3.8.12/envs/lewagon/lib/python3.8/site-packages/google/cloud/bigquery/table.py:2009: FutureWarning: Using .astype to convert from timezone-aware dtype to timezone-naive dtype is deprecated and will raise in a future version.  Use obj.tz_localize(None) or obj.tz_convert('UTC').tz_localize(None) instead
  df[column] = pandas.Series(df[column], dtype=dtypes[column])


200


In [10]:
df

,created_at,text,id,lang
0,2022-07-03 05:13:18+00:00,"#HTWknd | Meet @DaruvalaJehan, India’s only co...",1543462866745163777,en
2,2022-07-03 05:13:01+00:00,RT @F1: The last time a Spaniard took pole in ...,1543462794695266305,en
4,2022-07-03 05:12:58+00:00,RT @F1: Just the Medical Car doing a side whee...,1543462786025521152,en
5,2022-07-03 05:12:28+00:00,RT @VinuSquad: Who is waiting for $VINU to be ...,1543462658099363841,en
8,2022-07-03 05:12:21+00:00,RT @redbullracing: A new tyre-warming techniqu...,1543462629364174848,en
9,2022-07-03 05:12:21+00:00,RT @redbullracing: A new tyre-warming techniqu...,1543462626713374720,en
10,2022-07-03 05:12:20+00:00,RT @F1: Here's how they line up at Silverstone...,1543462625274642432,en
11,2022-07-03 05:12:14+00:00,RT @Motorsport: Sainz is now the 2️⃣nd Spanish...,1543462601379565570,en
12,2022-07-03 05:12:14+00:00,RT @JuliaF1fan: @prudydear Is this where we’re...,1543462599521607680,en
13,2022-07-03 05:12:05+00:00,ITS RACE DAY!! \n#BritishGP #F1 #F2 #F3,1543462560481087488,en
